# Introduing $\texttt{NatPy}$, a simple and convenient python module for dealing with and converting natural units.
## Requires $\texttt{python>=3.7 }$, $\texttt{astropy}$

In [1]:
!pip install natpy

### Ever wanted to convert MeV / fm$^{3}$ to Pascals?

In [2]:
import natpy as nat
a = 1 * nat.MeV * nat.fm**(-3)
a.convert(nat.Pa)

<Quantity 1.60217663e+32 Pa>

### Standard HEP unit conventions require 
> $\bar{h} = c = \epsilon_0=1$ 